<a href="https://colab.research.google.com/github/SKam23/10315-Final-Project/blob/main/10315_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 10-315 ML Final Project
Created by Shaheer Aslam, Steven Kam, and Sajan Shah.

< explanation of project here >

## Library Imports

In [231]:
import importlib
import torch 
import torchvision.transforms as transforms
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import torchvision.transforms.functional as TF
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from PIL import Image
from torchvision.utils import make_grid
from IPython.display import display

if importlib.util.find_spec("kaggle") is None:
    !pip install -q kaggle

# CelebA Dataset Processing

In [232]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/MyDrive/Colab Notebooks/final project/')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [233]:
# load images into a dictionary for easier access
images = {}
for filename in os.listdir('data/selected_images/'):
    if filename.endswith('.jpg'):
        image = Image.open(os.path.join('data/selected_images/', filename))
        images[filename] = image


In [234]:
print(images)

{'100926.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B67949BD0>, '103164.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B67949840>, '102797.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6794A140>, '102508.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6794AE60>, '102267.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6794A830>, '102247.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B67948970>, '102958.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B67949630>, '102767.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B679499F0>, '103001.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6794B430>, '103866.jpg': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=178x218 at 0x7F1B6794AA40>, '103132.jpg': <PIL.JpegImageP

In [235]:
# list_eval_partition.csv: Recommended partitioning of images into training, validation, testing sets. Images 1-162770 are training, 162771-182637 are validation, 182638-202599 are testing
# list_bbox_celeba.csv: Bounding box information for each image. "x_1" and "y_1" represent the upper left point coordinate of bounding box. "width" and "height" represent the width and height of bounding box
# list_landmarks_align_celeba.csv: Image landmarks and their respective coordinates. There are 5 landmarks: left eye, right eye, nose, left mouth, right mouth
# list_attr_celeba.csv: Attribute labels for each image. There are 40 attributes. "1" represents positive while "-1" represents negative
partition_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_eval_partition2000Smile.csv')
bbox_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_bbox_celeba2000Smile.csv')
landmarks_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_landmarks_align_celeba2000Smile.csv')
attr_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/final project/data/list_attr_celeba2000Smile.csv')



In [236]:
merged_df = partition_df.merge(bbox_df, on='image_id').merge(landmarks_df, on='image_id').merge(attr_df, on='image_id')
train_df = merged_df[merged_df['partition'] == 0]
val_df = merged_df[merged_df['partition'] == 1]
test_df = merged_df[merged_df['partition'] == 2]


In [237]:
train_df

,image_id,partition,x_1,y_1,width,height,lefteye_x,lefteye_y,righteye_x,righteye_y,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,Young
0,000219.jpg,0,291,78,122,169,68,110,107,113,...,-1,-1,-1,1,-1,-1,-1,-1,-1,1
1,000269.jpg,0,142,104,148,205,68,111,109,112,...,-1,1,-1,1,1,-1,1,-1,-1,1
2,000368.jpg,0,120,49,96,133,69,111,108,112,...,-1,1,-1,-1,-1,-1,-1,-1,-1,1
3,000428.jpg,0,60,67,107,148,68,111,109,112,...,-1,1,-1,-1,-1,-1,-1,-1,1,1
4,000555.jpg,0,72,20,122,169,68,112,109,112,...,1,1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1608,162387.jpg,0,87,94,166,230,69,110,107,112,...,-1,1,-1,1,1,-1,1,-1,-1,1
1609,162461.jpg,0,181,53,242,335,69,112,108,111,...,-1,-1,1,-1,-1,-1,1,-1,-1,1
1610,162624.jpg,0,188,159,203,281,68,112,109,112,...,-1,1,-1,1,-1,-1,1,-1,-1,1
1611,162656.jpg,0,189,114,361,500,70,111,107,111,...,-1,-1,-1,-1,1,-1,1,-1,-1,1


In [238]:
class CelebADataset(Dataset):
    def __init__(self, df, images):
        self.df = df
        self.images = images
        self.transform = transforms.Compose([transforms.Resize((178, 178)),transforms.ToTensor(),])
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_id = row['image_id']
        image = self.images[image_id]
        image = self.transform(image)
        is_smiling = row["Smiling"] == 1
        label = 1 if is_smiling else 0
        return image,label
        # return image, (label, image_id)

In [239]:
train_dataset = CelebADataset(train_df, images)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

val_dataset = CelebADataset(val_df, images)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=True)

test_dataset = CelebADataset(test_df, images)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [240]:
row = train_dataset.df.iloc[0]
row["Smiling"]

-1

In [241]:
import random

def get_random_images(dataset):
    smile_images = []
    not_smile_images = []
    
    for i in range(len(dataset)):
        image, label = train_dataset.__getitem__(i)
        if label[0] == 1:
            smile_images.append(dataset.images[label[1]])
        else:
            not_smile_images.append(dataset.images[label[1]])
    
    print(len(smile_images))
    print(len(not_smile_images))
    smile_image = random.choice(smile_images)
    not_smile_image = random.choice(not_smile_images)

    
    return smile_image, not_smile_image


In [242]:
# young_image, not_young_image = get_random_images(train_dataset)

# # Create a figure and set title
# fig = plt.figure(figsize=(8, 4))
# fig.suptitle('Smiling vs Not Smiling Images', fontsize=16)

# # Add young image to subplot 1
# ax1 = fig.add_subplot(1, 2, 1)
# ax1.imshow(young_image)
# ax1.set_title('Smiling Image')

# # Add not young image to subplot 2
# ax2 = fig.add_subplot(1, 2, 2)
# ax2.imshow(not_young_image)
# ax2.set_title('Not Smiling Image')

# plt.show()

# Convolutional Neural Network
< explanation >

In [249]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 44 * 44, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        return x

In [251]:
def evalDL(model, dl, lossfn):

    # figure out if the GPU is available and send model and data to it
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # use torch.no_grad() context manager to prevent gradients from accumulating in evaluation 
    # and hindering training
    with torch.no_grad():
        lossSum = 0
        nCorrect = 0
        for x, y in dl:
            x = x.to(device)
            y = y.to(device)

            yhat = model(x)

            # compute the loss and add to total (multiply by batch-size to make mean into sum)
            # by default, loss functions compute means of losses over batches
            lossSum += lossfn(yhat, y) * x.shape[0]

            # compute the number of correct and update
            predictions = torch.where(yhat > 0.5, 1, 0)
            truth = torch.where(y > 0.5, 1, 0)
            nCorrect += torch.sum(predictions == truth)

        numSamples = len(dl.dataset)
        avgLoss = lossSum / numSamples
        acc = nCorrect / numSamples
    return avgLoss.item(), acc.item()

def evaluate(model, trainDL, validDL, lossfn):
    trainLoss, trainAcc = evalDL(model, trainDL, lossfn)
    validLoss, validAcc = evalDL(model, validDL, lossfn)
    return trainLoss, trainAcc, validLoss, validAcc



In [260]:
import torch.nn.functional as F

def train(model, train_dataloader, val_dataloader, num_epochs):
    # define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())

    train_losses, val_losses = [], []
    train_accs, val_accs = [], []

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    for epoch in range(num_epochs):
        # training loop
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        for inputs, targets in train_dataloader:
          optimizer.zero_grad()
          inputs = inputs.to(device)
          targets = targets.to(device)
          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          targets_onehot = F.one_hot(targets, num_classes=2).float().to(device)  # convert targets to one-hot format
          loss = criterion(outputs, targets)
          loss.backward()
          optimizer.step()
          train_loss += loss.item() * inputs.size(0)
          train_total += targets.size(0)
          train_correct += (predicted == targets).sum().item()
        train_loss /= len(train_dataloader.dataset)
        train_losses.append(train_loss)
        train_acc = train_correct / train_total
        train_accs.append(train_acc)

        # validation loop
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for inputs, targets in val_dataloader:
                inputs = inputs.to(device)
                targets = targets.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                loss = criterion(outputs, targets)
                val_loss += loss.item() * inputs.size(0)
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()
            val_loss /= len(val_dataloader.dataset)
            val_losses.append(val_loss)
            val_acc = val_correct / val_total
            val_accs.append(val_acc)

        # print progress
        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')

    return train_losses, val_losses, train_accs, val_accs


In [262]:
train(CNN(), train_dataloader,val_dataloader,20)

Epoch [1/20], Train Loss: 1.0516, Train Acc: 0.5170, Val Loss: 0.6880, Val Acc: 0.5272
Epoch [2/20], Train Loss: 0.6909, Train Acc: 0.4954, Val Loss: 0.6893, Val Acc: 0.5272
Epoch [3/20], Train Loss: 0.6762, Train Acc: 0.5511, Val Loss: 0.6841, Val Acc: 0.5870
Epoch [4/20], Train Loss: 0.5925, Train Acc: 0.6869, Val Loss: 0.6203, Val Acc: 0.7120
Epoch [5/20], Train Loss: 0.4895, Train Acc: 0.7768, Val Loss: 0.5844, Val Acc: 0.6902
Epoch [6/20], Train Loss: 0.4233, Train Acc: 0.8146, Val Loss: 0.4463, Val Acc: 0.8098
Epoch [7/20], Train Loss: 0.3373, Train Acc: 0.8487, Val Loss: 0.4433, Val Acc: 0.7717
Epoch [8/20], Train Loss: 0.2825, Train Acc: 0.8797, Val Loss: 0.3908, Val Acc: 0.8478
Epoch [9/20], Train Loss: 0.2372, Train Acc: 0.9039, Val Loss: 0.4128, Val Acc: 0.8533
Epoch [10/20], Train Loss: 0.2135, Train Acc: 0.9138, Val Loss: 0.4491, Val Acc: 0.7880
Epoch [11/20], Train Loss: 0.2040, Train Acc: 0.9169, Val Loss: 0.3574, Val Acc: 0.8533
Epoch [12/20], Train Loss: 0.1611, Train 

([1.0516434058741022,
  0.6909473328138093,
  0.6761963030084349,
  0.5925213767738721,
  0.4894725690698003,
  0.4233387417476959,
  0.3372710138300673,
  0.28245023036605443,
  0.2372129275166168,
  0.21345139900424506,
  0.2040341065460972,
  0.16113348122309692,
  0.1451171240633641,
  0.12966429702221646,
  0.11708241517286602,
  0.10049862516218287,
  0.08570943452228467,
  0.07322133201193491,
  0.0650370067282264,
  0.06524500601404731],
 [0.6879877914553103,
  0.689312774202098,
  0.6841127405995908,
  0.6203290094500002,
  0.5843892071558081,
  0.4462668131227079,
  0.4433319063290306,
  0.39076070293136267,
  0.4127702622309975,
  0.4491330553656039,
  0.3573610523472662,
  0.3724505616270978,
  0.39304646072180377,
  0.3728612415168596,
  0.40022502126901044,
  0.4057176566642264,
  0.40006969026897266,
  0.41731109826461127,
  0.454296447660612,
  0.46293607354164124],
 [0.5170489770613763,
  0.495350278983261,
  0.5511469311841289,
  0.6869187848729076,
  0.77681339119652

In [ ]:
# def train(model, dataloader, criterion, optimizer, device):
#     model.train()
#     running_loss = 0.0
#     for i, (inputs, labels) in enumerate(dataloader):
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels[:, 0])
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#     return running_loss / len(dataloader)


# def test(model, dataloader, criterion, device):
#     model.eval()
#     running_loss = 0.0
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for i, (inputs, labels) in enumerate(dataloader):
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = criterion(outputs, labels[:, 0])
#             running_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels[:, 0]).sum().item()
#     return running_loss / len(dataloader), correct / total


# Training and Testing

In [ ]:
# # creating model
# model = CNN()
# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model.to(device)

In [ ]:
test(model, test_dataloader)